# CME 241 Assignment 3

## Shaan Patel

### Question 1

When using a deterministic policy, the action $a$ is given by a fixed function $\pi (s)$ that depends on the state. In this situation, there is no randomness in the action decision, as the state determines what action will be taken.

As such, the 4 MDP Bellman Policy equations become the following:

$ V^{\pi_D}(s) = R(s, \pi_D(s)) + \gamma \sum_{s' \in N} P(s, \pi_D(s), s') V^{\pi_D}(s') $

$ V^{\pi_D}(s) = Q^{\pi_D}(s, \pi_D(s))$

$Q^{\pi_D}(s, \pi_D(s)) = R(s, \pi_D(s)) + \gamma \sum_{s' \in N} P(s, \pi_D(s), s') V^{\pi_D}(s') $

$Q^{\pi_D}(s, \pi_D(s)) = R(s, \pi_D(s)) + \gamma \sum_{s' \in N}P(s, \pi(s), s') Q^{\pi_D}(s', \pi_D(s')) $

As shown, the $ \sum_{a \in A}$ gets removed because for each $s$ there is a fixed action that is taken based on $\pi(s)$.

### Question 2

Note that the rewards, transition probabilities, and states do not depend on the explicit value of $s$. As a result, we can see that the explicit value of $s$ is irrelevant to the value function, thus implying that the value function is the same for all values of $s$. Thus, $V^*(s) = V^*(s + 1)$, and

$$V^*(s) = \max_{a \in A} [R(s,a) + \gamma V^*(s) \sum_{s' \in N} P(s,a,s')] $$

The sum over all states of $P(s,a,s')$ is 1, so we get

$$V^*(s) = \max_{a \in A} [R(s,a) + \gamma V^*(s)] $$

This causes $ \gamma V^*(s)$ to not depend on $a$, so it can be removed from the max equation.

$$ (1 - \gamma)V^*(s) = \max_{a \in A} R(s,a) $$

$$ R(s,a) = E[r_{t+1}|S_t = s, A_t = a] = (1 - a)a + (1+a)(1-a) $$
$$ = (1-a)(2a + 1) = 1 + a - 2a^2 $$

The values of $a$ are either 0 or 1, thus $\max_{a \in A} R(s,a) = 1 $, when $a$ is 0. Thus,

$$ V^*(s) = \frac{1}{1 - \gamma} = 2$$

To have an optimal deterministic policy, we should hit the optimal value function for each state. As a result, because the value function is the same for all states, the optimal choice for $a$ remains the same across all $s$. Thus,

$$ \pi^*(s) = 0 $$

### Question 3

The state space of this frog problem is all lilypads from 0 to $n$.
The action space is the set $A, B$
For the transition function, we have

if $s' = s - 1$, 
$$P(s,a,s') = \frac{s + 1}{2n} $$

if $s' = s + 1$,
$$P(s,a,s') = \frac{n-s + 1}{2n}$$

all other cases,
$$P(s,a,s') = \frac{1}{2n} $$

In this scenario, the frog has an equal chance of choosing either action, and thus there is a 1/2 chance of choosing either croak. In the case of croak A, the frog can only go one space forward or backward. Thus there is no way to go to any state aside from $i-1$ or $i+1$ given croak A. If the frog croaks B, then there is a $\frac{1}{n}$ chance it can land one space away from its previous state as well, so these are included in the calculation.

### Question 4

The Expected Discounted Sum of Costs can be modeled through the MDP Bellman Policy Equation

$$V^\pi(s) = \sum_{a \in A} \pi(s,a) [R(s,a) + \gamma \sum_{s' \in N} P(s,a,s') V^\pi(s')] $$

In the myopic case, $\gamma$ is 0, so the equation simplifies to 

$$V^\pi(s) = \sum_{a \in A} \pi(s,a) R(s,a) $$

We are in the continuous case, so the sum is replaced with an integral, giving us

$$V^\pi(s) = \int_{A} \pi(s,a) R(s,a) da$$

The reward function $R$ is the expected return in the next period given current state $s$ and action $a$. Thus, because $s'$ follows a $N(s,\sigma)$ distribution, we can see that

$$ R(s,a) = E[e^{as'}] = \int_{\infty}^{\infty} e^{ax} \frac{1}{\sigma \sqrt{2\pi}} e^{\frac{-1}{2}(\frac{x - s}{\sigma})^2} dx$$

In the exponents, we have $2\sigma^2 ax - x^2 + 2xs - s^2 $ so we can complete the square to get $$(-x^2 + 2x(s + \sigma^2 a) - (s + \sigma^2 a)^2) + 2\sigma^2 as +  \sigma^4 a^2 $$ or $$-(x - s - \sigma^2 a)^2 + 2\sigma^2 as + \sigma^4 a^2  $$

Because the latter two terms have no x, we can take them out of the integral

$$ e^{\frac{\sigma^2 a (2s + \sigma^2 a)}{2\sigma^2}} \int_\infty^\infty \frac{1}{\sigma \sqrt{2\pi}} e^{\frac{-1}{2} (\frac{x - s - \sigma^2 a}{\sigma})^2} dx$$

The integral thus simplifies to the pdf of a $N(s + \sigma^2 a, \sigma)$ distribution, so integrating it over all real values equates it to 1. So,

$$R(s,a) = e^{\frac{a (2s + \sigma^2 a)}{2}} $$

Going back to the value function we see that

$$V^\pi(s) = \int_A \pi(s,a) e^{\frac{a (2s + \sigma^2 a)}{2}} da$$

For any s, the minimum value of this function (provided that $\pi(s,a)$ is 1 at this minimum value and 0 everywhere else) is when $a(2s + \sigma^2 a)$ is at its minimum. Therefore, the minimum is reached when $2s + 2\sigma^2 a= 0$, or

$$ a = \frac{-s}{\sigma^2} $$

This provides a cost of $e^{\frac{-ss'}{\sigma^2}}$